# Group 29 M3 Submission

In [179]:
import pandas as pd
from math import exp
import numpy as np
import math
from itertools import permutations, combinations, product
from functools import reduce
from scipy.optimize import linprog

Parameters for Group 29

$u = 1.1 + \frac{29}{100} = 1.39$

$d = \frac{1}{u} = 0.7194$

$p^* = \frac{1 - d}{u - d} = 0.4184$


In [180]:
S = 95  # stock underlying value
K = 105 # strike
t = 1.0 # time
r = 0.0 # risk-free interest rate
n = 3 # number of steps
v = 0.0 # volatility
T = t * n
u = 1.39
d = 1 / u
At = t / n 
p = (1 - d) / (u - d) 

## Binomial Tree generating functions

In [181]:
def buildtree_xvalues(u, d, T, S):
    
    # Generate the paths
    a=product('ud', repeat=T) 
    paths = []
    for i in a:
        paths.append(''.join(i))
    
    # Calculate the paths
    b=product([u,d], repeat=T) 
    vals= []
    for i in b:
        vals.append(list(i))
    vals1 = np.matrix(vals)
    
    # Create table for the paths
    # generate first column
    startingvals = np.repeat(S, 2**T)
    rv = startingvals.reshape(2**T,1)
    # generate subsequent columns by multiplying the values of previous columns
    stockvalues = []
    stockvalues.append(rv.T.tolist()[0])
    for i in range(T):
        rv = np.multiply(rv, vals1[:,i])
        stockvalues.append(rv.T.tolist()[0])   
    
    # Create a dataframe from the matrix of path values
    stockvalues = np.array(stockvalues)
    dfstock = pd.DataFrame(stockvalues.T)

    dfstock.index = paths
    dfstock = dfstock.applymap(lambda x: np.round(x,2))
    return dfstock    


def buildtree_Call(u, d, T, S, K, cutoff=True):
    
    dfstock = buildtree_xvalues(u, d, T, S)
    
    # Calc option values. Otherwise just return the X values
    if cutoff:
        # (X_t - K)+ payoff
        dfstock['H'] = dfstock[T].apply(lambda x: x-K if x-K > 0 else 0)
    else:
        # (X_t - K) payoff
        dfstock['H'] = dfstock[T]-K
    return dfstock


def buildtree_Put(u, d, T, S, K, cutoff=True):
    
    dfstock = buildtree_xvalues(u, d, T, S)
   
    # Calculate the last column which is the payoff
    if cutoff:
        dfstock['H'] = dfstock[T].apply(lambda x: K-x if x-K < 0 else 0)
    else:
        dfstock['H'] = K-dfstock[T]
    return dfstock


In [182]:
def pi_h_star(df: pd.DataFrame, p: list):
    # if lengths of EMM and paths do not match then return error code -1
    if len(p) != len(df):
        return False
    valh = np.round(np.sum(df['H']*p),2)
    return valh

def iscomplete(df: pd.DataFrame, p: list):
    # if lengths of EMM and paths do not match then return error code -1
    if len(p) != len(df):
        return False
    val0 = np.round(np.sum(df[df.columns[0]]*p),2)
    valh = np.round(np.sum(df['H']*p),2)
    return val0 == valh


def pi_h(u, d, df, col=None, K=None):

    if col is None or col == 'H':
        hvals = df['H'].to_list()
    else:
        if col not in df.columns:
            return np.nan
        else:
            hvals = df[col].apply(lambda x: x-K if x-K < 0 else 0).to_list() 
                
    # Calculate the p*
    p = (1 - d) / (u - d) 
    q = 1 - p
    pieH = 0
    T = int(np.log2(len(df)))
    
    # get the product structure p^m q^n
    t1 = perm(p, q, T)
    # reshape the H values to perform matrix multiplication
    h = np.round(hvals, 2)
    h2 = h.reshape(len(h), 1)
    prod1 = np.multiply(t1, h2)
    # unique no-arbitrage price of H is
    pieH = np.sum(prod1)
    return pieH

    

### EMM

In [183]:
def perm(p, q, T):
    b=product((p,q), repeat=T) 
    vals= []
    for i in b:
        vals.append(list(i))
    vals1 = np.matrix(vals)
    # print(vals1)
    t1 = reduce(lambda a,b: np.multiply(a,b), [m for m in vals1.T])
    return t1.T

In [184]:
def emm1(df):
    u = df[1].max()/df[0].max()
    d = df[1].min()/df[0].max()
    p = (1 - d) / (u - d) 
    q = 1 - p
    # print(p,q)
    # get the product structure p^m q^n
    t1 = perm(p, q, len(df.columns))
    return t1
    

def emm2(df, a=0.01):
    obj = [0.001]*len(df)
    bnd = [(a, float("inf"))]*len(df)
    lhs_eq = df.T.to_numpy()
    rhs_eq = [df[0].iloc[0]]*len(df.columns)
    opt = linprog(c=obj,A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,method="revised simplex")
    p_star = np.round(opt.x, 6)
    return p_star


def emm3(df):

    # if paths = T
    if len(df) == len(df.columns):
        a = df.T.to_numpy()
        b = df[0].to_numpy()*1.00
        p_star = np.linalg.solve(a, b)
        if np.any(p_star > 0):
            return p_star
        else:
            return []
    # if paths = T+1
    elif len(df) == len(df.columns) + 1:
        a = df.T.to_numpy()
        b = df[0].to_numpy()*1.00
        lastrow = np.zeros(len(df1))
        lastrow[0] = 1
        a = np.append(a, [lastrow], axis = 0)
        b = df[0].to_numpy()*1.00
        for i in np.arange(0.1,1,0.1):
            b[-1] = i
            p_star = np.linalg.solve(a, b)
            if np.all(p_star > 0):
                return p_star
    # if paths > T+1
    elif len(df) > len(df.columns) + 1:
        df_emm = df_h.sample(n=T+2)
        a = df_emm.T.to_numpy()
        b = df_emm[0].to_numpy()*1.00
        lastrow = np.zeros(len(df_emm))
        lastrow[0] = 1
        a = np.append(a, [lastrow], axis = 0)
        b = df_emm[0].to_numpy()*1.00
        print(a)
        print(b)
        for i in np.arange(0.1,1,0.1):
            b[-1] = i
            # 
            print(i)
            solns = np.linalg.solve(a, b)
            if np.all(solns > 0):
                df_emm['p*'] = solns
                df_temp = df.copy()
                df_temp['p*']  = 0
                for idx in df_emm.index:
                    df_temp.loc[idx, 'p*'] = df_emm.loc[idx, 'p*']
                p_star = df_temp['p*'].to_list()
                return p_star
    else:
        return []
    return []


def emm4(df, df_h):

    # if paths = T
    if len(df) == len(df.columns):
        a = df.T.to_numpy()
        b = df[0].to_numpy()*1.00
        p_star = np.linalg.solve(a, b)
        if np.any(p_star > 0):
            return p_star
    # if paths = T+1
    elif len(df) == len(df.columns) + 1:
        a = df.T.to_numpy()
        b = df[0].to_numpy()*1.00

        # add last 2 columns from option values to have an extra contrain to add to the system of linear equations
        df3 = df_h[df_h.columns[-2:]]
        lastrow = df_h[df_h.columns[-2:]].diff(axis=1).T.dropna().iloc[0].to_numpy()
        # print(a)
        # print(lastrow)

        b[-1] = 0.0
        a = np.append(a, [lastrow], axis = 0)
        # print(a)
        # print(b)
        
        p_star = np.linalg.solve(a, b)
        p_star = np.round(p_star, 6)
        print(p_star)
        if np.all(p_star >= 0):
            return p_star
    return [1/len(df)]*len(df)

In [185]:
def isEMM(df: pd.DataFrame, p: list):
    # if lengths of EMM and paths do not match then return error code -1
    if len(p) != len(df):
        return False
    val = np.round(np.sum(df[df.columns[0]]*p),2)
    for c in df.columns[1:]:
        valnew = np.round(np.sum(df[c]*p),2)
        if valnew != val:
            return False
    return True

In [186]:
def snell(df: pd.DataFrame, p: list, showEU=False):
    df_U = df.copy()
    l = len(df.columns)
    df_U['p'] = p
    for i in reversed(range(l-1)):
        df_U['mult'] = df_U['p']*df_U[i+1]
        df3 = df_U.groupby(i).sum()
        df3['E_U'] = df3['mult']/df3['p']
        df3.reset_index(inplace=True)
        df3['U'] = df3[['E_U', i]].max(axis=1)
        df_U[i] = [df3[df3[i] == r]['U'].iloc[0] for r in df_U[i]]
        if showEU:
            print(df_U)
            print(df3)
    df_U = df_U.drop(columns=['p', 'mult'])
    return df_U

In [187]:
def tau_0(df_U, df_h):
    stoppingtime = (df_h >= df_U) & (df_U > 0)
    stoppingtime = stoppingtime*stoppingtime.columns
    stoppingtime = stoppingtime[stoppingtime > 0].T.min()
    stoppingtime = stoppingtime.fillna(df_h.columns[-1])
    return stoppingtime

# Week 7 Group Assignment M3

In [188]:
u=1.39
d=1/u
T=5
S=95
K=90
p = (1 - d) / (u - d) 
q = 1 - p

In [189]:
df_x = buildtree_xvalues(u, d, T, S)
df_x

,0,1,2,3,4,5
uuuuu,95.0,132.05,183.55,255.13,354.64,492.94
uuuud,95.0,132.05,183.55,255.13,354.64,255.13
uuudu,95.0,132.05,183.55,255.13,183.55,255.13
uuudd,95.0,132.05,183.55,255.13,183.55,132.05
uuduu,95.0,132.05,183.55,132.05,183.55,255.13
uudud,95.0,132.05,183.55,132.05,183.55,132.05
uuddu,95.0,132.05,183.55,132.05,95.00,132.05
uuddd,95.0,132.05,183.55,132.05,95.00,68.35
uduuu,95.0,132.05,95.00,132.05,183.55,255.13
uduud,95.0,132.05,95.00,132.05,183.55,132.05


## Part 1:  Call Option

In [190]:
df_h = df_x - K
df_h = df_h.applymap(lambda x: 0 if x < 0 else x)
df_h

,0,1,2,3,4,5
uuuuu,5.0,42.05,93.55,165.13,264.64,402.94
uuuud,5.0,42.05,93.55,165.13,264.64,165.13
uuudu,5.0,42.05,93.55,165.13,93.55,165.13
uuudd,5.0,42.05,93.55,165.13,93.55,42.05
uuduu,5.0,42.05,93.55,42.05,93.55,165.13
uudud,5.0,42.05,93.55,42.05,93.55,42.05
uuddu,5.0,42.05,93.55,42.05,5.00,42.05
uuddd,5.0,42.05,93.55,42.05,5.00,0.00
uduuu,5.0,42.05,5.00,42.05,93.55,165.13
uduud,5.0,42.05,5.00,42.05,93.55,42.05


In [191]:
p_star = emm2(df_h)
p_star

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.69])

In [192]:
isEMM(df_h, p_star)

False

In [193]:
df_U = snell(df_h, p, showEU=False)
df_U

,0,1,2,3,4,5
uuuuu,51.534063,81.577969,128.060000,193.812500,284.035,402.94
uuuud,51.534063,81.577969,128.060000,193.812500,284.035,165.13
uuudu,51.534063,81.577969,128.060000,193.812500,103.590,165.13
uuudd,51.534063,81.577969,128.060000,193.812500,103.590,42.05
uuduu,51.534063,81.577969,128.060000,62.307500,103.590,165.13
uudud,51.534063,81.577969,128.060000,62.307500,103.590,42.05
uuddu,51.534063,81.577969,128.060000,62.307500,21.025,42.05
uuddd,51.534063,81.577969,128.060000,62.307500,21.025,0.00
uduuu,51.534063,81.577969,35.095938,62.307500,103.590,165.13
uduud,51.534063,81.577969,35.095938,62.307500,103.590,42.05


In [194]:
stoppingtime= tau_0(df_U, df_h)
stoppingtime

uuuuu    5.0
uuuud    5.0
uuudu    5.0
uuudd    5.0
uuduu    5.0
uudud    5.0
uuddu    5.0
uuddd    5.0
uduuu    5.0
uduud    5.0
ududu    5.0
ududd    5.0
udduu    5.0
uddud    5.0
udddu    5.0
udddd    5.0
duuuu    5.0
duuud    5.0
duudu    5.0
duudd    5.0
duduu    5.0
dudud    5.0
duddu    5.0
duddd    5.0
dduuu    5.0
dduud    5.0
ddudu    5.0
ddudd    5.0
ddduu    5.0
dddud    5.0
ddddu    5.0
ddddd    5.0
dtype: float64

### Early exercise benefit
There is no early exercise benefit.


### Price of option

In [195]:
df3 = df_h.copy()
df3['H'] = df_h[df_h.columns[-1]]
df3

,0,1,2,3,4,5,H
uuuuu,5.0,42.05,93.55,165.13,264.64,402.94,402.94
uuuud,5.0,42.05,93.55,165.13,264.64,165.13,165.13
uuudu,5.0,42.05,93.55,165.13,93.55,165.13,165.13
uuudd,5.0,42.05,93.55,165.13,93.55,42.05,42.05
uuduu,5.0,42.05,93.55,42.05,93.55,165.13,165.13
uudud,5.0,42.05,93.55,42.05,93.55,42.05,42.05
uuddu,5.0,42.05,93.55,42.05,5.00,42.05,42.05
uuddd,5.0,42.05,93.55,42.05,5.00,0.00,0.00
uduuu,5.0,42.05,5.00,42.05,93.55,165.13,165.13
uduud,5.0,42.05,5.00,42.05,93.55,42.05,42.05


In [196]:
pi_h(u, d, df3)

30.3027367705053

## Part 2: Put Option

In [225]:
df_h = K - df_x
df_h = df_h.applymap(lambda x: x if x > 0 else 0)
df_h

,0,1,2,3,4,5
uuuuu,0,0.00,0.00,0.00,0.00,0.00
uuuud,0,0.00,0.00,0.00,0.00,0.00
uuudu,0,0.00,0.00,0.00,0.00,0.00
uuudd,0,0.00,0.00,0.00,0.00,0.00
uuduu,0,0.00,0.00,0.00,0.00,0.00
uudud,0,0.00,0.00,0.00,0.00,0.00
uuddu,0,0.00,0.00,0.00,0.00,0.00
uuddd,0,0.00,0.00,0.00,0.00,28.06
uduuu,0,0.00,0.00,0.00,0.00,0.00
uduud,0,0.00,0.00,0.00,0.00,0.00


In [226]:
p_star = emm2(df_h)
p_star

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

In [227]:
isEMM(df_h, p_star)

False

In [228]:
df_U = snell(df_h, p, showEU=False)
df_U

,0,1,2,3,4,5
uuuuu,22.901364,14.455152,14.455152,7.652727,7.652727,0.00
uuuud,22.901364,14.455152,14.455152,7.652727,7.652727,0.00
uuudu,22.901364,14.455152,14.455152,7.652727,7.652727,0.00
uuudd,22.901364,14.455152,14.455152,7.652727,7.652727,0.00
uuduu,22.901364,14.455152,14.455152,7.652727,7.652727,0.00
uudud,22.901364,14.455152,14.455152,7.652727,7.652727,0.00
uuddu,22.901364,14.455152,14.455152,7.652727,7.652727,0.00
uuddd,22.901364,14.455152,14.455152,7.652727,7.652727,28.06
uduuu,22.901364,14.455152,14.455152,7.652727,7.652727,0.00
uduud,22.901364,14.455152,14.455152,7.652727,7.652727,0.00


In [229]:
stoppingtime= tau_0(df_U, df_h)
stoppingtime

uuuuu    5.0
uuuud    5.0
uuudu    5.0
uuudd    5.0
uuduu    5.0
uudud    5.0
uuddu    5.0
uuddd    5.0
uduuu    5.0
uduud    5.0
ududu    5.0
ududd    5.0
udduu    3.0
uddud    3.0
udddu    3.0
udddd    3.0
duuuu    5.0
duuud    5.0
duudu    5.0
duudd    5.0
duduu    3.0
dudud    3.0
duddu    3.0
duddd    3.0
dduuu    2.0
dduud    2.0
ddudu    2.0
ddudd    2.0
ddduu    2.0
dddud    2.0
ddddu    2.0
ddddd    2.0
dtype: float64

There is early exercise benefits for the following events.

In [230]:
stoppingtime[stoppingtime<5]

udduu    3.0
uddud    3.0
udddu    3.0
udddd    3.0
duduu    3.0
dudud    3.0
duddu    3.0
duddd    3.0
dduuu    2.0
dduud    2.0
ddudu    2.0
ddudd    2.0
ddduu    2.0
dddud    2.0
ddddu    2.0
ddddd    2.0
dtype: float64

## Part 3: Exotic option

In [203]:
u=1.39
d=1/u
T=5
S=100
K=100
p = (1 - d) / (u - d) 
q = 1 - p
df_x = buildtree_xvalues(u, d, T, S)
df_x

,0,1,2,3,4,5
uuuuu,100.0,139.00,193.21,268.56,373.30,518.89
uuuud,100.0,139.00,193.21,268.56,373.30,268.56
uuudu,100.0,139.00,193.21,268.56,193.21,268.56
uuudd,100.0,139.00,193.21,268.56,193.21,139.00
uuduu,100.0,139.00,193.21,139.00,193.21,268.56
uudud,100.0,139.00,193.21,139.00,193.21,139.00
uuddu,100.0,139.00,193.21,139.00,100.00,139.00
uuddd,100.0,139.00,193.21,139.00,100.00,71.94
uduuu,100.0,139.00,100.00,139.00,193.21,268.56
uduud,100.0,139.00,100.00,139.00,193.21,139.00


#### UAO European call option values

In [204]:
barrier = 130
df_h_uao = df_x.applymap(lambda x: 0 if x > barrier else x)

df_h_uao = df_h_uao - K
df_h_uao = df_h_uao.applymap(lambda x: 0 if x < 0 else x)

df_h_uao

,0,1,2,3,4,5
uuuuu,0.0,0,0.0,0,0.0,0
uuuud,0.0,0,0.0,0,0.0,0
uuudu,0.0,0,0.0,0,0.0,0
uuudd,0.0,0,0.0,0,0.0,0
uuduu,0.0,0,0.0,0,0.0,0
uudud,0.0,0,0.0,0,0.0,0
uuddu,0.0,0,0.0,0,0.0,0
uuddd,0.0,0,0.0,0,0.0,0
uduuu,0.0,0,0.0,0,0.0,0
uduud,0.0,0,0.0,0,0.0,0


All values of the Up-and-Out (UAO) European Call Option is 0 since there is only payoff if the price is between 100 and 130 at maturity (T=5). However, in our binomial tree the prices never fall between 100 and 130.

#### Vanilla European Call Option

In [205]:
u=1.39
d=1/u
T=5
S=95
K=90
p = (1 - d) / (u - d) 
q = 1 - p
df_x = buildtree_xvalues(u, d, T, S)
df_x
df_h = df_x - K
df_h = df_h.applymap(lambda x: 0 if x < 0 else x)
df_h

,0,1,2,3,4,5
uuuuu,5.0,42.05,93.55,165.13,264.64,402.94
uuuud,5.0,42.05,93.55,165.13,264.64,165.13
uuudu,5.0,42.05,93.55,165.13,93.55,165.13
uuudd,5.0,42.05,93.55,165.13,93.55,42.05
uuduu,5.0,42.05,93.55,42.05,93.55,165.13
uudud,5.0,42.05,93.55,42.05,93.55,42.05
uuddu,5.0,42.05,93.55,42.05,5.00,42.05
uuddd,5.0,42.05,93.55,42.05,5.00,0.00
uduuu,5.0,42.05,5.00,42.05,93.55,165.13
uduud,5.0,42.05,5.00,42.05,93.55,42.05


In [206]:
p_star = emm2(df_h, a=0.001)
p_star

array([0.001   , 0.001   , 0.001   , 0.001   , 0.001   , 0.029754,
       0.013546, 0.001   , 0.007794, 0.001   , 0.001   , 0.044104,
       0.012708, 0.001   , 0.001   , 0.001   , 0.001   , 0.001   ,
       0.001   , 0.001   , 0.001   , 0.001   , 0.001   , 0.001   ,
       0.001   , 0.001   , 0.001   , 0.001   , 0.001   , 0.001   ,
       0.001   , 0.866094])

In [207]:
df3 = df_h.copy()
df3['H'] = df_h[df_h.columns[-1]]
df3

,0,1,2,3,4,5,H
uuuuu,5.0,42.05,93.55,165.13,264.64,402.94,402.94
uuuud,5.0,42.05,93.55,165.13,264.64,165.13,165.13
uuudu,5.0,42.05,93.55,165.13,93.55,165.13,165.13
uuudd,5.0,42.05,93.55,165.13,93.55,42.05,42.05
uuduu,5.0,42.05,93.55,42.05,93.55,165.13,165.13
uudud,5.0,42.05,93.55,42.05,93.55,42.05,42.05
uuddu,5.0,42.05,93.55,42.05,5.00,42.05,42.05
uuddd,5.0,42.05,93.55,42.05,5.00,0.00,0.00
uduuu,5.0,42.05,5.00,42.05,93.55,165.13,165.13
uduud,5.0,42.05,5.00,42.05,93.55,42.05,42.05


In [208]:
pi_h(u, d, df3)

30.3027367705053

3.a. The value of the Up-and-Out (UAO) European Call Option is 0 since there is only payoff if the price is between 100 and 130 at maturity (T=5). However, in our binomial tree the prices at expiration do not fall between 100 and 130.

3.b. The price of the European call is 30.30 while the price of UAO European call is 0. Therefore, the European call is more expensive.

3.c. The advantage of the up-and-out European call option is that is cheaper than vanilla European call option. Since there is only a narrow band where the UAO option is worth something, the corresponding payoff probabilibity is low and so is the expectation value of the option.

In [209]:
barrier = 130
df_h_uai = df_x.applymap(lambda x: 0 if x < barrier else x)

df_h_uai = df_h_uai - K
df_h_uai = df_h_uai.applymap(lambda x: 0 if x < 0 else x)

df_h_uai

,0,1,2,3,4,5
uuuuu,0,42.05,93.55,165.13,264.64,402.94
uuuud,0,42.05,93.55,165.13,264.64,165.13
uuudu,0,42.05,93.55,165.13,93.55,165.13
uuudd,0,42.05,93.55,165.13,93.55,42.05
uuduu,0,42.05,93.55,42.05,93.55,165.13
uudud,0,42.05,93.55,42.05,93.55,42.05
uuddu,0,42.05,93.55,42.05,0.00,42.05
uuddd,0,42.05,93.55,42.05,0.00,0.00
uduuu,0,42.05,0.00,42.05,93.55,165.13
uduud,0,42.05,0.00,42.05,93.55,42.05


In [210]:
p_star = emm2(df_h_uai)
p_star

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

## Part 4: Market completeness

In [211]:
def phi(lastvals: list, hvals: list):
    obj = [0.001]*2
    bnd = [(-float("inf"), float("inf"))]*2
    lhs_eq = np.array([np.ones(len(lastvals)), lastvals]).T
    rhs_eq = hvals
    opt = linprog(c=obj,A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,method="revised simplex")
    phi_vals = np.round(opt.x, 6)
    return phi_vals

def iscomplete(lastvals: list, hvals: list):
    phi_vals = phi(lastvals, hvals)
    return np.any(phi_vals != 0)

In [212]:
u=1.39
d=1/u
T=5
S=95
K=90
p = (1 - d) / (u - d) 
q = 1 - p

df_x = buildtree_xvalues(u, d, T, S)

df_h = df_x - K
df_h = df_h.applymap(lambda x: 0 if x < 0 else x)

In [213]:
lastvals = df_x[5].to_list()
hvals = df_h[5].to_list()
phi(lastvals, hvals)

/tmp/ipykernel_30609/671487115.py:6: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  opt = linprog(c=obj,A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,method="revised simplex")


array([0., 0.])

4.a. The market is NOT complete when using binomial tree for American Call and Put options.

In [214]:
u=1.39
d=1/u
T=5
S=100
K=100
p = (1 - d) / (u - d) 
q = 1 - p
df_x = buildtree_xvalues(u, d, T, S)
barrier = 130
df_h_uao = df_x.applymap(lambda x: 0 if x > barrier else x)

df_h_uao = df_h_uao - K
df_h_uao = df_h_uao.applymap(lambda x: 0 if x < 0 else x)

df_h_uao


,0,1,2,3,4,5
uuuuu,0.0,0,0.0,0,0.0,0
uuuud,0.0,0,0.0,0,0.0,0
uuudu,0.0,0,0.0,0,0.0,0
uuudd,0.0,0,0.0,0,0.0,0
uuduu,0.0,0,0.0,0,0.0,0
uudud,0.0,0,0.0,0,0.0,0
uuddu,0.0,0,0.0,0,0.0,0
uuddd,0.0,0,0.0,0,0.0,0
uduuu,0.0,0,0.0,0,0.0,0
uduud,0.0,0,0.0,0,0.0,0


4.b. The market is NOT complete when using binomial tree for European UAO call option.

4.c. Market is complete when H is attainable. This means there must be a solution for $ X \cdot \phi = V = H \Rightarrow A \cdot \phi = H$. If the rank of the matrix A.  

Let A be an mxn matrix. The rank of A is the maximum number of linearly independent row vectors. If the rank is m then the $\phi$ are linearly independent. If the rank is less than m, then the vectors are linearly dependant. When the $\phi$  are linearly dependent then there is no solution and the market is not complete. 